# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

Для начала мы ипортируем пакеты, импортируем данные, посмотрим на их распределение

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import lightgbm as lgb
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder 
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
import time 
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('/datasets/autos.csv')

In [4]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


Уже можно заметить очень странные значения в некоторых столбцах, перечислю отдельно:
1) Цена вряд ли может быть меньше нуля, очень нелогично, думаю стоит убрать нулевые значения

2) Год регистрации не может быть меньше 1899 года, только в этом году начали регистрировать автомобили, также как и не может быть больше 2023

3) Мощность равная нулю очень странна, не может быть меньше 1, также как и не может быть больше 2000 лс.

4) С осталными столбцами все хорошо (машина также не может быть зарегестрирована в 0 месяце, также что это тоже странно, но это вряд ли может на что то повлиять)

Мне я это показалось странным, я это заметил, скорее всего это ошибка сбора данных, так как данный столбец отвечает за количество фотографий объекта

Давайте уберем тогда, согласен

In [5]:
data = data.drop('NumberOfPictures', axis = 1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  PostalCode         354369 non-null  int64 
 14  LastSeen           354369 non-null  object
dtypes: int64(6), object(9)
memory usage: 40.6+ MB


In [32]:
data.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


На этом этапе можно удалить 2 столбца, а именно дату размещения объявления и последний просмотр, так как они в себе по сути не несут никакой информации

In [33]:
data = data.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis = 1)

In [34]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [35]:
data = data.query('Price > 0 & RegistrationYear > 1899 & RegistrationYear < 2023 & Power > 0 & Power < 2000')

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307084 entries, 1 to 354368
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              307084 non-null  int64 
 1   VehicleType        285739 non-null  object
 2   RegistrationYear   307084 non-null  int64 
 3   Gearbox            301002 non-null  object
 4   Power              307084 non-null  int64 
 5   Model              294618 non-null  object
 6   Kilometer          307084 non-null  int64 
 7   RegistrationMonth  307084 non-null  int64 
 8   FuelType           287301 non-null  object
 9   Brand              307084 non-null  object
 10  Repaired           259961 non-null  object
 11  NumberOfPictures   307084 non-null  int64 
 12  PostalCode         307084 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 32.8+ MB


Мы убрали всех аутлайеров, значений было 354000 а стало 307000, что не является критичным. Теперь поработаем над пропусками и дубликатами

Хорошо, принял, тогда предлагаю их вообще не удалять, так как их всего несколько строк и они мало на что повлияют

Дубликаты удалили успешно, удалилось всего несколько строк, далее можно просто удалить все пропуски, конечно, их достаточно много, но в любом случае у нас останется более 200000 значений

In [42]:
data.isnull().sum()

Price                   0
VehicleType            24
RegistrationYear        0
Gearbox              6082
Power                   0
Model                  23
Kilometer               0
RegistrationMonth       0
FuelType               37
Brand                   0
Repaired               70
NumberOfPictures        0
PostalCode              0
dtype: int64

Для начала попробуем восстановить тип кузова исходя из мощности, понятно, что есть множество разных, но тем не менее обычно у конкретной модели конкретное количество мощности и лошадиных сил, условно у тех же малолитражен может быть от 90 до 120 лошадиных сил в зависимости от конкретного производителя, с фуел тайпом сделаем тоже самое, как и с моделью, что касается ремонта, то какие то двигатели ремнтируются чаще, какие то реже, поэтому это также можно привязать к мощности

In [43]:
data['VehicleType'] = data['VehicleType'].fillna(data.groupby('Power')['VehicleType'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
data['Model'] = data['Model'].fillna(data.groupby('Power')['Model'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
data['FuelType'] = data['FuelType'].fillna(data.groupby('Power')['FuelType'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
data['Repaired'] = data['Repaired'].fillna(data.groupby('Power')['Repaired'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
data['Gearbox'] = data['Gearbox'].fillna(data.groupby('Power')['Gearbox'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))

In [44]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306969 entries, 1 to 354368
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              306969 non-null  int64 
 1   VehicleType        306969 non-null  object
 2   RegistrationYear   306969 non-null  int64 
 3   Gearbox            306969 non-null  object
 4   Power              306969 non-null  int64 
 5   Model              306969 non-null  object
 6   Kilometer          306969 non-null  int64 
 7   RegistrationMonth  306969 non-null  int64 
 8   FuelType           306969 non-null  object
 9   Brand              306969 non-null  object
 10  Repaired           306969 non-null  object
 11  NumberOfPictures   306969 non-null  int64 
 12  PostalCode         306969 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 32.8+ MB


Данных осталось гораздо больше

Также чтобы у нас верно работало машинное обучение нужно превратить признаки типа object в categorical, но это я сделаю после деления датасета

После небольшой первичной обработки данных можно начать обучать модели и работать с этими данными

Тут напишу что было сделано с данными:

    1. Проверены аутлайеры а также просто ненормальные и нелогичные значения
    
    2. Были удалены такие значения
    
    3. Были удалены дубликаты
    
    4. Были обработаны пропуски

## Обучение моделей

Для начала разделим датасет на обучающую и тестовую выборки, также выделим таргет переменную и признаки.

In [46]:
target = data['Price']
features = data.drop('Price', axis = 1)
for col in features.columns:
    col_type = features[col].dtype
    if col_type == 'object' or col_type.name == 'category':
        features[col] = features[col].astype('category')
features_train, features_valid1, target_train, target_valid1 = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid1, target_valid1, test_size=0.5, random_state=12345)

In [18]:
start = time.time() 
param_grid = {
    'class_weight': [None, 'balanced'],
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': list(range(30, 150)),
    'learning_rate': [0.01,0.1,0.5],
    'subsample_for_bin': [20000,50000,100000,120000,150000],
    'min_child_samples': [20,50,100,200,500],
    'colsample_bytree': [0.6,0.8,1],
    "max_depth": [5,10,50,100]
}

lgbm = LGBMRegressor() 
lgbm.fit(features_train, target_train) 

lgbm_tuned = LGBMRegressor(boosting_type = 'gbdt',
                            class_weight = None,
                            min_child_samples = 20,
                            num_leaves = 30,
                            subsample_for_bin = 20000,
                            learning_rate=0.01,
                            max_depth=10,
                            n_estimators=40)
lgbm_tuned.fit(features_train, target_train)

y_test_pred = lgbm_tuned.predict(features_valid)
score = mean_squared_error(target_valid, y_test_pred, squared=False)
end = time.time() - start
print(score)
print(end)

Принял, поделю на 3 выборки

Результат получился не очень, попробуем еще кэтбуст и рандомный лес

In [47]:
start = time.time() 
cat_features = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'Repaired']

model_cat = CatBoostRegressor(iterations=50, depth = 10) 

model_cat.fit(features_train, target_train, cat_features=cat_features, verbose=10)

y_test_pred = model_cat.predict(features_valid)
end = time.time() - start
print(mean_squared_error(target_valid, y_test_pred, squared=False))
print(end)

Learning rate set to 0.5
0:	learn: 3126.5149833	total: 182ms	remaining: 8.92s
10:	learn: 1781.8692743	total: 1.36s	remaining: 4.81s
20:	learn: 1689.7064764	total: 2.41s	remaining: 3.33s
30:	learn: 1628.2375603	total: 3.52s	remaining: 2.16s
40:	learn: 1579.0137787	total: 4.56s	remaining: 1s
49:	learn: 1548.6744910	total: 5.55s	remaining: 0us
1683.8770013372055
9.178460597991943


Используя другую библиотеку для бустинга получился результат гораздо лучше, RMSE всего 1600. Теперь построим рандомный лес, также применим ван хот енкодинг, чтобы все работало

In [47]:
data1 = pd.get_dummies(data)
data1.head()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode,VehicleType_bus,VehicleType_convertible,VehicleType_coupe,...,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,Repaired_no,Repaired_yes
3,1500,2001,75,150000,6,0,91074,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,3600,2008,69,90000,7,0,60437,0,0,0,...,1,0,0,0,0,0,0,0,1,0
5,650,1995,102,150000,10,0,33775,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2200,2004,109,150000,8,0,67112,0,1,0,...,0,0,0,0,0,0,0,0,1,0
10,2000,2004,105,150000,12,0,96224,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [66]:
target = data1['Price']
features = data1.drop('Price', axis = 1)
features_train, features_valid1, target_train, target_valid1 = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid1, target_valid1, test_size=0.5, random_state=12345)

In [71]:
start = time.time() 
best_RMSE = 10000
for est in range(1, 101, 20):
    for depth in range(1, 17, 4):
                model = RandomForestRegressor(n_estimators = est, max_depth=depth, random_state=12345)
                model.fit(features_train, target_train)
                y_test_pred = model.predict(features_valid)
                result_RMSE = mean_squared_error(target_valid, y_test_pred, squared=False)
                if result_RMSE < best_RMSE:
                    best_RMSE = result_RMSE
end = time.time() - start
print(best_RMSE)
print(end)

1736.8492163067604


Получился еще один очень хороший результат, очень близкий к кэтбустингу, люблю рандомный лес!

In [67]:
start = time.time() 
model = LinearRegression()
model.fit(features_train, target_train)
y_test_pred = model.predict(features_valid)
result_RMSE = mean_squared_error(target_valid, y_test_pred, squared=False)
end = time.time() - start
print(result_RMSE)
print(end)

34.537431478500366


В линейной регрессии результат получился неплохой, но все равно хуже чем в кэтбусте и радомном лесу

## Анализ моделей

Анализируя все имеющиеся модели самый лучший RMSE получился именно у catboost'а, там он равен 1600, это очень хороший результат, модель обучается достаточно быстро, что также является достойным результатом.

Чтобы не быть головловным сделаем предсказания по лучшей модели на тестовой выборке. Что касается времени то кэтбуст также оказлась самой быстрой библиотекой.

Обучим ее еще раз и проверим.

In [69]:
target = data['Price']
features = data.drop('Price', axis = 1)
for col in features.columns:
    col_type = features[col].dtype
    if col_type == 'object' or col_type.name == 'category':
        features[col] = features[col].astype('category')
features_train, features_valid1, target_train, target_valid1 = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid1, target_valid1, test_size=0.5, random_state=12345)

In [70]:
start = time.time() 
cat_features = ['VehicleType', 'Gearbox', 'Model',
                'FuelType', 'Brand', 'Repaired']

model_cat = CatBoostRegressor(iterations=50, depth = 10) 

model_cat.fit(features_train, target_train, cat_features=cat_features, verbose=10)

y_test_pred = model_cat.predict(features_valid)
end = time.time() - start
print(mean_squared_error(target_valid, y_test_pred, squared=False))
print(end)

Learning rate set to 0.5
0:	learn: 3138.7125087	total: 87.1ms	remaining: 4.27s
10:	learn: 1713.7630837	total: 908ms	remaining: 3.22s
20:	learn: 1615.6439363	total: 1.8s	remaining: 2.49s
30:	learn: 1562.2652301	total: 2.67s	remaining: 1.64s
40:	learn: 1513.0835306	total: 3.52s	remaining: 772ms
49:	learn: 1484.4462215	total: 4.24s	remaining: 0us
1611.1574972351932
5.682898283004761


In [71]:
y_pred = model_cat.predict(features_test)
print(mean_squared_error(target_test, y_pred, squared=False))

1591.1538610589882


Результат

Модель catboost'а оказалась самой быстрой и точной моделью, время обучения составляет всего 8-10 секунд. По сравнению с другими моделями, например линейной регрессией, которая обычается все 40 секунд это очень быстро.

Точность также очень высокая, RMSE всего лишь 1600, ближайшая к ней 1700 в раномном лесу, который обучается ну очень долго

В результате мы получилио обученную модель, которая может предсказывать цену автомобиля.

За этот проект было сделано следующее:

    1. Проверены аутлайеры а также просто ненормальные и нелогичные значения
    
    2. Были удалены такие значения
    
    3. Были удалены дубликаты
    
    4. Были обработаны пропуски
    
    5. Был проведен one-hot-encoding
    
    6. Построена и обучена модель через библиотеку lightGBM
    
    7. Построена и обучена модель через библиотеку CatBoost
    
    8. Построена и обучена модель с помощью рандомного леса
    
    9. Построена и обучена модель с помощью обычной линейной регрессии
    
Результаты были получены достаточно хорошие, модель предсказывает достаточно четко